In [ ]:
import community as community_louvain
from collections import defaultdict
from functools import partial
import sklearn.metrics
import networkx as nx
from glob import glob
import igraph as ig
import pandas as pd
import numpy as np
import re
import os

In [ ]:
# Make dirs for plots
for algorithm in ['Leiden', 'Louvain', 'Multi-level', 'Fast-greedy']:
    if not os.path.exists(algorithm):
        os.makedirs(algorithm)

In [ ]:
metrics = defaultdict(list)
networks = []

for path in glob("../real/*.net"):
    g = ig.load(path).simplify()
    
    reference_path = path.replace(".net", ".clu")
    if os.path.exists(reference_path):
        with open(reference_path, 'r') as f:
            next(f)
            reference = [int(line)-1 for line in f]
            reference = ig.VertexClustering(g, reference)
    else:
        continue
    name = os.path.basename(path)[:-4]
    networks.append(name)
    
    layout = g.layout_auto()
    #plot = partial(ig.plot, layout=layout, bbox=(300, 300), margin=20, mark_groups=True)
    plot = partial(ig.plot, layout=layout)

    #LEIDEN
    leiden = g.community_leiden('modularity', n_iterations=-1)
    plot(leiden, f"Leiden/{name}.png")
    
    #LOUVAIN
    louvain = community_louvain.best_partition(g.to_networkx())
    louvain = ig.VertexClustering(g, louvain.values())
    plot(louvain, f"Louvain/{name}.png")
    
    #MULTI-LEVEL
    multilevel = g.community_multilevel()
    plot(multilevel, f"Multi-level/{name}.png")
    
    #FAST GREEDY
    fastgreedy = g.community_fastgreedy().as_clustering()
    plot(fastgreedy, f"Fast-greedy/{name}.png")
    
    communities = {
        'leiden': leiden,
        'louvain': louvain,
        'multilevel': multilevel,
        'fastgreedy': fastgreedy,
        'reference': reference
    }
    
    metric_methods = {
        'vi': partial(ig.compare_communities, reference.membership, method='vi'),
        'nmi': partial(ig.compare_communities, reference.membership, method='nmi'),
        'jaccard': partial(sklearn.metrics.jaccard_score, reference.membership, average='weighted'),
    }
    
    for community_name, community in communities.items():
        metrics[community_name + '_mod'].append(community.modularity)
        if community_name == 'reference':
            break
        for metric_name, metric in metric_methods.items():
            metrics[community_name + '_' + metric_name].append(metric(community.membership))
#SAVE TO TABLE
df = pd.DataFrame(metrics, index=networks)
print("\nMODULARITY TABLE\n", df.filter(regex=".*_mod"))
print("\nVI TABLE\n", df.filter(regex=".*_vi"))
print("\nNMI TABLE\n", df.filter(regex=".*_nmi"))
print("\nJACCARD TABLE\n", df.filter(regex=".*_jaccard"))